In [ ]:
# Kod snutt till labb_1 om jag hellre vill plotta som en funktion

def plot_category_vs_cardio(df, category, title, ax, rotate_xticks=False, palette=None):

    percent = df.groupby(category)['cardio'].mean() * 100
    sns.barplot(x=percent.index, y=percent.values, ax=ax, palette=palette)
    ax.set_title(title)
    ax.set_ylabel('Andel positiva (%)')
    if rotate_xticks:
        ax.set_xticks(range(len(percent.index)))  # Sätt antalet ticks
        ax.set_xticklabels(percent.index, rotation=45)

# Skapa en figur med subplots
plt.figure(figsize=(15, 12))

# Plot 1: Blodtryckskategorier
plt.subplot(2, 2, 1)
plot_category_vs_cardio(df, 'blood_pressure_category', 'Blodtryckskategorier vs Andel Positiva', 
                        plt.gca(), rotate_xticks=True, palette='viridis')

# Plot 2: BMI-kategorier
plt.subplot(2, 2, 2)
plot_category_vs_cardio(df, 'BMI_category', 'BMI-kategorier vs Andel Positiva', 
                        plt.gca(), rotate_xticks=True)

# Plot 3: Blodsocker
plt.subplot(2, 2, 3)
plot_category_vs_cardio(df_gluc, 'gluc_category', 'Blodsocker vs Andel Positiva', 
                        plt.gca())

# Plot 4: Alkohol
plt.subplot(2, 2, 4)
plot_category_vs_cardio(df, 'alco', 'Alkohol vs Andel Positiva', 
                        plt.gca())
plt.xticks(ticks=[0, 1], labels=['Inte alkoholkonsumtion', 'Alkoholkonsumtion'])


plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
# Train-validation-test split, punkt 1 i välj modell
# Dela upp df1
X_df1 = df1.drop('cardio', axis=1) # Feauters
y_df1 = df1['cardio'] # Target

X_train_df1, X_test_df1, y_train_df1, y_test_df1 = train_test_split(X_df1, y_df1, test_size=0.2, random_state=42)
X_train_df1, X_val_df1, y_train_df1, y_val_df1 = train_test_split(X_train_df1, y_train_df1, test_size=0.25, random_state=42)

# Dela upp df2
X_df2 = df2.drop('cardio', axis=1)
y_df2 = df2['cardio']

X_train_df2, X_test_df2, y_train_df2, y_test_df2 = train_test_split(X_df2, y_df2, test_size=0.2, random_state=42)
X_train_df2, X_val_df2, y_train_df2, y_val_df2 = train_test_split(X_train_df2, y_train_df2, test_size=0.25, random_state=42)

# Visa storleken på varje del
print("Storlekar för df1:")
print(f"Träningsdata: {X_train_df1.shape}, Valideringsdata: {X_val_df1.shape}, Testdata: {X_test_df1.shape}")
print("\nStorlekar för df2:")
print(f"Träningsdata: {X_train_df2.shape}, Valideringsdata: {X_val_df2.shape}, Testdata: {X_test_df2.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler
# Skala dataset punkt 2 i välj modell
scaler_standard = StandardScaler()
X_train_df1_std = scaler_standard.fit_transform(X_train_df1)
X_val_df1_std = scaler_standard.transform(X_val_df1)
X_test_df1_std = scaler_standard.transform(X_test_df1)

from sklearn.preprocessing import MinMaxScaler

scaler_minmax = MinMaxScaler()
X_train_df1_norm = scaler_minmax.fit_transform(X_train_df1)
X_val_df1_norm = scaler_minmax.transform(X_val_df1)
X_test_df1_norm = scaler_minmax.transform(X_test_df1)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline

# Funktion för att träna och evaluera Random Forest
def train_evaluate_rf(df):
    # Train-validation-test split
    X = df.drop('cardio', axis=1)
    y = df['cardio']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    
    # Skapa pipeline med skalning och modell
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RandomForestClassifier(random_state=42))
    ])
    
    # Definiera hyperparametrar
    param_grid = {
        'model__n_estimators': [10, 20, 30],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2]
    }
    
    # GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Prediction och evaluation
    y_pred = grid_search.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    # Returnera resultat
    return {
        'best_params': grid_search.best_params_,
        'accuracy': accuracy,
        'f1_score': f1
    }

# Kör Random Forest på båda dataseten
result_df1 = train_evaluate_rf(df1)
result_df2 = train_evaluate_rf(df2)

# Visa resultat
print("Resultat för df1:")
print(result_df1)
print("\nResultat för df2:")
print(result_df2)